In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split

In [2]:
def Channel_Shuffle(x,groups):
    g = groups
    batch, height, width, channels = x.shape
    x = tf.reshape(x, [-1, height, width, g, channels // g])
    x = tf.transpose(x, perm = [0, 1, 2, 4, 3])
    x = tf.reverse(x,[-1])
    x = tf.reshape(x, [-1, height, width, channels])
    return x

In [3]:
def Basic_Unit(x,f,groups):
    x1,x2=layers.Lambda(lambda x: tf.split(x,num_or_size_splits=2,axis=3))(x)

    x2=layers.Conv2D(filters=f/4,kernel_size=1,strides=1,padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.DepthwiseConv2D(kernel_size=3,strides=1,padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.Conv2D(filters=f,kernel_size=1,strides=1,padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)

    x3=layers.Concatenate()([x1,x2])
    x3=layers.Lambda(lambda x:Channel_Shuffle(x,groups=groups) )(x3)
    return x3


In [4]:
def Downsampling_Unit(x,f):
    x1=layers.DepthwiseConv2D(kernel_size=3,strides=2,padding='same')(x)
    x1=layers.BatchNormalization()(x1)
    x1=layers.Conv2D(filters=f,kernel_size=1,strides=1,padding='same')(x1)
    x1=layers.BatchNormalization()(x1)
    x1=layers.ReLU()(x1)

    x2=layers.Conv2D(filters=f/4,kernel_size=1,strides=1,padding='same')(x)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)
    x2=layers.DepthwiseConv2D(kernel_size=3,strides=2,padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.Conv2D(filters=f,kernel_size=1,strides=1,padding='same')(x2)
    x2=layers.BatchNormalization()(x2)
    x2=layers.ReLU()(x2)

    x3=layers.Concatenate()([x1,x2])
    x3=layers.Lambda(lambda x:Channel_Shuffle(x,4))(x3)
    return x3

In [8]:
input=Input(shape=(224,224,3))
x=layers.Conv2D(filters=24,kernel_size=3,strides=2,padding='same')(input)
x=layers.MaxPool2D(pool_size=3,strides=2,padding='same')(x)
print(x.shape)

x=Downsampling_Unit(x,116)
print(x.shape)

x=Basic_Unit(x,116,4)
x=Basic_Unit(x,116,4)
x=Basic_Unit(x,116,4)

x=Downsampling_Unit(x,232)

x=Basic_Unit(x,232,4)
x=Basic_Unit(x,232,4)
x=Basic_Unit(x,232,4)
x=Basic_Unit(x,232,4)
x=Basic_Unit(x,232,4)
x=Basic_Unit(x,232,4)
x=Basic_Unit(x,232,4)

x=Downsampling_Unit(x,464)

x=Basic_Unit(x,464,4)
x=Basic_Unit(x,464,4)
x=Basic_Unit(x,464,4)

x=layers.Conv2D(filters=1024,kernel_size=1,strides=1,padding='same')(x)
print(x.shape)
x=layers.GlobalAveragePooling2D()(x)
x=layers.Dense(units=1000,activation='softmax')(x)

model = tf.keras.models.Model(input,x)

(None, 56, 56, 24)
(None, 28, 28, 232)
(None, 7, 7, 1024)


In [9]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_76 (Conv2D)             (None, 112, 112, 24  672         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 56, 56, 24)  0           ['conv2d_76[0][0]']              
                                                                                            

In [ ]:
plot_model(model,to_file='ShuffleNet_v2.png',show_shapes=True)

In [17]:
dir=r'D:\dataset\butterfly\train'
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
dir,
labels="inferred",
label_mode="int",
class_names=None,
color_mode="rgb",
batch_size=4,
image_size=(224, 224),
shuffle=True,
seed=10,
validation_split=0.1,
subset='training',
interpolation="gaussian",
follow_links=False,
crop_to_aspect_ratio=False,)


validation_ds=tf.keras.preprocessing.image_dataset_from_directory(
dir,
labels="inferred",
label_mode="int",
class_names=None,
color_mode="rgb",
batch_size=4,
image_size=(224, 224),
shuffle=True,
seed=10,
validation_split=0.1,
subset='validation',
interpolation="gaussian",
follow_links=False,
crop_to_aspect_ratio=False,)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255.)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

Found 1257 files belonging to 10 classes.
Using 1132 files for training.
Found 1257 files belonging to 10 classes.
Using 125 files for validation.


In [19]:
#해당 모델은 lr=le-3으로 시작할 경우 gradient explode하는 것 같았음 그래서 lr 시작점을 1e-4로 잡아줌 그러니까 제대로된 acc 나옴 
lr_rate=tf.keras.optimizers.schedules.ExponentialDecay(1e-4, 10000, 0.97, staircase=False, name=None)

    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_rate),loss='sparse_categorical_crossentropy',metrics=['acc'])


In [22]:
history=model.fit(train_ds,validation_data=validation_ds,epochs=50,batch_size=8,verbose=1)

Epoch 1/50
283/283 [==============================] - 13s 45ms/step - loss: 1.9388 - acc: 0.3118 - val_loss: 3.3442 - val_acc: 0.1440
Epoch 2/50
283/283 [==============================] - 13s 44ms/step - loss: 1.6279 - acc: 0.3975 - val_loss: 1.1101 - val_acc: 0.5680
Epoch 3/50
283/283 [==============================] - 13s 45ms/step - loss: 1.4314 - acc: 0.5027 - val_loss: 1.0513 - val_acc: 0.6320
Epoch 4/50
283/283 [==============================] - 13s 44ms/step - loss: 1.3003 - acc: 0.5601 - val_loss: 0.8987 - val_acc: 0.7040
Epoch 5/50
283/283 [==============================] - 13s 44ms/step - loss: 1.2525 - acc: 0.5804 - val_loss: 0.7116 - val_acc: 0.7360
Epoch 6/50
283/283 [==============================] - 12s 44ms/step - loss: 1.1999 - acc: 0.6060 - val_loss: 0.7823 - val_acc: 0.7280
Epoch 7/50
283/283 [==============================] - 13s 44ms/step - loss: 1.0535 - acc: 0.6511 - val_loss: 0.8733 - val_acc: 0.7360
Epoch 8/50
283/283 [==============================] - 13s 44ms